In [1]:
import os
import pickle
import numpy as np
import scipy as sci
import glob
import mne
from autoreject import AutoReject
from mne.preprocessing import ICA
from mne_icalabel import label_components
from autoreject import get_rejection_threshold
from utills import cal_foof
import matplotlib.pyplot as plt

# Set figure resolution
plt.rcParams['figure.dpi'] = 600
plt.rcParams['savefig.dpi'] = 600

# Paths
data_path = '/home/hno/datasets/update_for_hamzeh/Python/IOWA_DATA/IOWA_Processing/Public_Datasets/Raw_data/'
save_root = '/home/hno/datasets/update_for_hamzeh/Python/IOWA_DATA/IOWA_Processing/results/'
# Groups
groups = ['pd', 'hc']

# AutoReject initialization
ar = AutoReject(cv=3, n_interpolate=[1, 4, 8, 16], consensus=[0.5, 1], random_state=313, n_jobs=-1,
                thresh_method='bayesian_optimization', verbose=False)

# Function for processing each group (Parkinson or Healthy)
def process_group(group, sbj_files, save_root):
    ica_rejected = dict()
    failed_subj = dict()
    bad_subj = dict()
    group_result = dict()

    for fmin, fmax in freqs:
        f_range = [fmin, fmax]
        save_fooof = os.path.join(save_root, f'Range_{fmin}-{fmax}_Hz')
        os.makedirs(save_fooof, exist_ok=True)
        print(f'{group.capitalize()} Group: Range {f_range} Started ....')

        for subj in sbj_files:
            name = subj.split('_Rest/')[-1].split('.vhdr')[0]
            raw = mne.io.read_raw_brainvision(subj, preload=True, verbose=False)

            # Apply filters and other preprocessing steps
            filtered = preprocess_data(raw)

            # ICA decomposition
            ica = ICA(n_components=len(filtered.ch_names) - 1, max_iter="auto", method="infomax", random_state=313)
            ica.fit(filtered.copy(), verbose=False)

            # Labeling ICA components and rejecting bad components
            exclude_idx = label_and_reject_ica(ica, filtered)

            if len(np.unique(exclude_idx)) == len(ica.labels_) or len(np.unique(exclude_idx)) > 0.8 * len(ica.labels_):
                print(f'All components rejected for Subj {name}')
                bad_subj[group] = subj
                continue

            # Apply ICA rejection
            raw_ica = ica.apply(filtered.copy(), exclude=np.unique(exclude_idx), verbose=False)
            epochs_ar = create_epochs(raw_ica)
            epochs_ar.save(os.path.join(save_fooof, f'{name}_epo.fif'), overwrite=True)

            # Store results
            group_result[name] = cal_foof(epochs_ar, f_range=f_range, low_freq_to_del=False, high_freq_to_del=False, remove_line_noise=False)

        # Saving results
        save_results(save_fooof, group, ica_rejected, failed_subj, bad_subj, group_result)

# Preprocessing data
def preprocess_data(raw):
    filtered = raw.copy().pick(picks="eeg").filter(l_freq=1, h_freq=100, picks=None, filter_length='auto',
                                                   l_trans_bandwidth='auto', h_trans_bandwidth='auto', n_jobs=-1,
                                                   method='iir', iir_params=None, phase='zero', pad='reflect_limited', verbose=False)
    montage = filtered.get_montage()
    filtered.set_montage(montage, match_case=False)
    filtered.apply_function(sci.signal.detrend, type='linear')
    filtered.set_eeg_reference(ref_channels='average', verbose=False)
    return filtered

# Labeling and rejecting ICA components
def label_and_reject_ica(ica, filtered):
    try:
        ica_labels = label_components(filtered, ica, method='iclabel')
        labels = ica_labels["labels"]
        exclude_idx = [idx for idx, label in enumerate(labels) if label not in ["brain"]]
        # if label in ["brain"] and ica_labels['y_pred_proba'][idx] < 0.7: # optional
            #exclude_idx.append(idx)
    except:
        print(f"ICA labeling failed")
        return []

    return exclude_idx

# Creating epochs
def create_epochs(raw_ica):
    epochs = mne.make_fixed_length_epochs(raw_ica, duration=1, overlap=0, preload=True, verbose=False)
    epochs.apply_function(sci.signal.detrend, type='linear')
    reject_criteria = get_rejection_threshold(epochs)
    epochs.drop_bad(reject=reject_criteria, verbose=False)
    ar.fit(epochs)
    epochs_ar, rejected_log = ar.fit_transform(epochs, return_log=True)
    epochs_ar.pick(['eeg'])
    return epochs_ar

# Saving results
def save_results(save_fooof, group, ica_rejected, failed_subj, bad_subj, group_result):
    with open(os.path.join(save_fooof, f'failed_subj.pkl'), 'wb') as file:
        pickle.dump(failed_subj[group], file)

    with open(os.path.join(save_fooof, 'bad_subj.pkl'), 'wb') as file:
        pickle.dump(bad_subj[group], file)

    with open(os.path.join(save_fooof, f'{group.capitalize()}_rejected_ICs.pkl'), 'wb') as file:
        pickle.dump(ica_rejected[group], file)

    with open(os.path.join(save_fooof, f'{group}.pkl'), 'wb') as file:
        pickle.dump(group_result, file)

# Get subject files
files = glob.glob(os.path.join(data_path, '**/*.vhdr'), recursive=True)
sbj_files = sorted([file for file in files if '.vhdr' in file])
parkinson = [file for file in sbj_files if 'PD' in file]
healthy = [file for file in sbj_files if 'Con' in file]

# Process both groups
process_group('pd', parkinson, save_root)
process_group('hc', healthy, save_root)



In [ ]:


# Define conditions and file paths
condition = ['PD vs HC', 'Parkinson', 'Control']
save_root = '/path/to/your/dataset/IOWA_Processing'
data_path = '/path/to/your/dataset/IOWA_Processing/results'

# Frequency ranges for analysis
freqs = [
    [1, 20], [1, 30], [1, 40], [1, 50], [1, 60], [1, 70], [1, 80], [1, 90], [3, 20], [3, 30], [3, 40], [3, 50], [3, 60], [3, 70], [3, 80], [3, 90]
]

# Loop through the frequency ranges for analysis
for freq in freqs:
    fmin, fmax = freq
    f_range = [fmin, fmax]

    # Directory to save the processed data for this frequency range
    save_fooof = os.path.join(save_root, f'Range_{fmin}-{fmax}_Hz')
    os.makedirs(save_fooof, exist_ok=True)
    print(f'Starting analysis for frequency range {f_range} Hz...')

    # Load Parkinson's and Control data
    with open(os.path.join(data_path, 'parkinson.pkl'), 'rb') as f:
        parkinson_data = pickle.load(f)

    with open(os.path.join(data_path, 'healthy.pkl'), 'rb') as f:
        control_data = pickle.load(f)

    # Calculate accuracy for the Control group
    acc_control = [1 - control_data[name]['AllScalp'].error_ for name in control_data]

    # Calculate accuracy for the Parkinson's group
    acc_parkinson = [1 - parkinson_data[name]['AllScalp'].error_ for name in parkinson_data]

    # Calculate Z-scores for outlier detection
    z_score_PD = [abs((x - np.mean(acc_parkinson)) / np.std(acc_parkinson)) for x in acc_parkinson]
    z_score_CT = [abs((x - np.mean(acc_control)) / np.std(acc_control)) for x in acc_control]

    # Identify outliers with Z-score > 2
    bad_index_PD = [x for x, z in enumerate(z_score_PD) if z > 2]
    bad_index_CT = [x for x, z in enumerate(z_score_CT) if z > 2]

    # Remove outliers from both Parkinson's and Control data
    parkinson_data_cleaned = {k: v for i, (k, v) in enumerate(parkinson_data.items()) if i not in bad_index_PD}
    control_data_cleaned = {k: v for i, (k, v) in enumerate(control_data.items()) if i not in bad_index_CT}

    # Save cleaned data
    with open(os.path.join(save_fooof, 'Healthy_group.pkl'), 'wb') as f:
        pickle.dump(control_data_cleaned, f)

    with open(os.path.join(save_fooof, 'Parkinson_group.pkl'), 'wb') as f:
        pickle.dump(parkinson_data_cleaned, f)

    # Generate signal plots and extract frequency features
    results = signal_plot(parkinson_data_cleaned, control_data_cleaned, save_fooof, condition=['', '', ''])
    peak_index = get_area(results, f_range, save_root=save_fooof, srate=500)

    # Define regions and features of interest
    regions = ['Frontal', 'Central', 'Occipital', 'Left', 'Right', 'AllScalp']
    features = ['ID', 'acc', 'exp', 'offs', 'alpha-CF', 'alpha-peak', 'beta-CF', 'beta-peak']

    brain_features = {'PD': {region: {feature: [] for feature in features} for region in regions},
                      'CT': {region: {feature: [] for feature in features} for region in regions}}

    # Populate brain features for Parkinson's group
    for region in regions:
        for name in parkinson_data_cleaned:
            brain_features['PD'][region]['ID'].append(name)
            brain_features['PD'][region]['exp'].append(np.round(parkinson_data_cleaned[name][region].get_params('aperiodic_params')[1], 4))
            brain_features['PD'][region]['offs'].append(np.round(parkinson_data_cleaned[name][region].get_params('aperiodic_params')[0], 4))
            brain_features['PD'][region]['acc'].append(np.round(1 - parkinson_data_cleaned[name][region].error_, 4))

        brain_features['PD'][region]['alpha-CF'] = list(peak_index['parkinson'][region]['alpha']['CF'])
        brain_features['PD'][region]['alpha-peak'] = list(peak_index['parkinson'][region]['alpha']['peak'])
        brain_features['PD'][region]['beta-CF'] = list(peak_index['parkinson'][region]['beta']['CF'])
        brain_features['PD'][region]['beta-peak'] = list(peak_index['parkinson'][region]['beta']['peak'])

    # Populate brain features for Control group
    for region in regions:
        for name in control_data_cleaned:
            brain_features['CT'][region]['ID'].append(name)
            brain_features['CT'][region]['exp'].append(np.round(control_data_cleaned[name][region].get_params('aperiodic_params')[1], 4))
            brain_features['CT'][region]['offs'].append(np.round(control_data_cleaned[name][region].get_params('aperiodic_params')[0], 4))
            brain_features['CT'][region]['acc'].append(np.round(1 - control_data_cleaned[name][region].error_, 4))

        brain_features['CT'][region]['alpha-CF'] = list(peak_index['healthy'][region]['alpha']['CF'])
        brain_features['CT'][region]['alpha-peak'] = list(peak_index['healthy'][region]['alpha']['peak'])
        brain_features['CT'][region]['beta-CF'] = list(peak_index['healthy'][region]['beta']['CF'])
        brain_features['CT'][region]['beta-peak'] = list(peak_index['healthy'][region]['beta']['peak'])

    # Create a DataFrame from the extracted features
    rows = []
    for group, regions in brain_features.items():
        for region, features in regions.items():
            length = len(parkinson_data_cleaned) if group == 'PD' else len(control_data_cleaned)
            for i in range(length):
                row = {
                    'ID': features['ID'][i],
                    'Group': group,
                    'Region': region,
                    'acc': features['acc'][i],
                    'exp': features['exp'][i],
                    'offs': features['offs'][i],
                    'alpha-CF': np.round(features['alpha-CF'][i], 4),
                    'alpha-peak': features['alpha-peak'][i],
                    'beta-CF': np.round(features['beta-CF'][i], 4),
                    'beta-peak': features['beta-peak'][i]
                }
                rows.append(row)

    df = pd.DataFrame(rows, columns=['ID', 'Group', 'Region', 'acc', 'exp', 'offs', 'alpha-CF', 'alpha-peak', 'beta-CF', 'beta-peak'])

    # Save the DataFrame as CSV
    df.to_csv(os.path.join(save_fooof, 'data.csv'), index=False)

    print(f'Analysis for frequency range {f_range} Hz completed.')
